In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df1 = pd.read_pickle('./data/12-15_at_1816est_sortby_hot_use_10comms.pkl')
df2 = pd.read_pickle('./data/12-15_at_1805est_sortby_new_use_10comms.pkl')
df3 = pd.read_pickle('./data/12-15_at_1805est_sortby_rising_use_10comms.pkl')

In [3]:
df = pd.concat([df1, df2, df3])
df = df[~df.index.duplicated(keep='first')]

In [4]:
print('There are {} submissions.'.format(df.shape[0]))

There are 1388 submissions.


In [5]:
df.head(3)

,Title,Author,ID,Gilded,Gildings,Upvotes,Upvote ratio,Post time,Views,Discussion type,...,Subreddit,Post age,Upvote rate,Comment rate,Avg top comments up rate,Std top comments up rate,Gilded rate,Distinguished rate,Op comment rate,Premium author rate
kdstuu,Nestle illegally fires worker for reporting ra...,fallingleaf_271,kdstuu,0,{'gid_1': 2},26690,0.94,764,None,None,...,news,212.261,125.741,4.50388,0.119903,0.0251586,0,0,0,0
kdqntm,High-Elf Joe Biden,Link3265,kdqntm,0,{'gid_1': 3},37352,0.91,657,None,None,...,pics,318.504,117.273,2.28569,0.0293813,0.00113704,0.2,0,0,0.4
kdq1p2,TIL Frank Sinatra died the night of Seinfeld's...,sunghooter,kdq1p2,0,{'gid_1': 7},76288,0.92,627,None,None,...,todayilearned,349.323,218.388,3.66996,0.17064,0.07367,0.2,0,0,0.2


Remove any entries which have already been gilded.

Users may interact with already-gilded posts differently then yet-to-be-gilded posts, best to drop any posts which have been gilded so as not to see any confounding effects.

In [6]:
reddit = df[df['Gilded'] == 0]

Drop those columns which either aren't used, or don't contain usable information.

In [7]:
cols_to_drop = ['Title',
                'Author',
                'ID',
                'Gilded',
                'Views',
                'Discussion type',
                'Hidden',
                'Crosspostable',
                'Robot indexable',
                'Upvotes',
                'Likes',
                'Comments',
                'Post age',
               ]
reddit = reddit.drop(columns = cols_to_drop)

Extract from the 'Gildings' columns those posts which have been awarded silver, create a new column to hold this info and then drop the 'Gildings' column.

In [8]:
reddit['Num silver'] = reddit['Gildings'].apply(lambda x: x['gid_1'] if not x=={} else 0)
reddit = reddit.drop(columns = ['Gildings'])

Remove nulls from 'Distinguished' feature by replacing them with the string 'False' to create a new (third) category.

In [9]:
reddit['Distinguished'].fillna(value = 'False', inplace = True)

Content categories is only used for a handful of subreddits. Extract those categories when present otherwise return 'None'.

In [10]:
reddit['Content categories'] = reddit['Content categories'].apply(lambda x: 'None' if x is None else x[0])

Those submissions which have been edited are marked by their edit time. Replace these with the boolean True as I will not use the exact time of edit, rather keeping only a flag as to whether an edit has been made or not.

In [11]:
reddit['Edited'] = reddit['Edited'].apply(lambda x: True if x != False else x)

Cast the 'Crossposts' feature as an integer, counting the number of crossposts.

In [12]:
reddit['Crossposts'] = reddit['Crossposts'].astype(int)

Consider seperately those features which are categorical vs. those that are numerical.

In [13]:
categorical = ['Distinguished',
               'Contest mode',
               'Content categories',
               'Edited',
               'Meta',
               'OC',
               'Reddit media',
               'Selfpost',
               'Video',
               'Adult content',
               'Subreddit',
              ]
numerical = ['Upvote ratio',
             'Post time',
             'Crossposts',
             'Upvote rate',
             'Comment rate',
             'Avg top comments up rate',
             'Std top comments up rate',
             'Gilded rate',
             'Distinguished rate',
             'Op comment rate',
             'Premium author rate',
             'Num silver',
            ]

In [15]:
reddit_cat = reddit[categorical]
reddit_num = reddit[numerical]

print('Total number of categorical plus numerical: ', len(categorical)+len(numerical))
print('Total number of columns: ', reddit.shape[1])
print('\nTotal number of usable submissions {}'.format(reddit.shape[0]))

Total number of categorical plus numerical:  23
Total number of columns:  23

Total number of usable submissions 1352


In [23]:
df.loc[df['Post age'].sort_values(ascending = False).index[0],:]



Title                       Today we’re testing a new way to discuss polit...
Author                                                                   spez
ID                                                                     ipitt0
Gilded                                                                      3
Gildings                                            {'gid_1': 15, 'gid_2': 3}
Upvotes                                                                 16003
Upvote ratio                                                             0.57
Post time                                                                 606
Views                                                                    None
Discussion type                                                          None
Distinguished                                                           admin
Contest mode                                                            False
Content categories                                              